# Data types

## tuple

### create
```
a = tuple(range(10))
```

### slice

`a[start:end:step]`

## list

### create
```
a = [i for i in range(10)]
b = list((1, 2, 3, 4))
```
### slice

`a[start:end:step]`

```
a[::2]
a[1:3]
a[:3]
a[-2:]
```

## dict

### create
```
a = {'name': 'Yang Zongze', 'id': 1234, 'department': 'AMA'}
```

### get item
```
a['name']
a.get('age', 30)
```

```
for k, v in a.items():
    print(k, v)
```

In [ ]:
a = {'name': 'Yang Zongze', 'id': 1234, 'department': 'AMA'}
for k, v in a.items():
    print(k, v)

# Builtin functions

## `map`

In [ ]:
s = map(lambda x, y: x+y, [1, 2, 3], [4, 5, 6])
for i in s:
    print(i)

## `reduce`

In [ ]:
from functools import reduce
from operator import add, mul
from math import sin

# reduce(lambda x, y: x+y, [1, 2, 3, 4, 5])

# sin(1)*sin(2)*sin(3)
reduce(mul, map(sin, [1, 2, 3]))

# GC



Ref:

1. https://devguide.python.org/internals/garbage-collector/index.html
1. https://jakevdp.github.io/blog/2014/05/09/why-python-is-slow/
2. https://zhuanlan.zhihu.com/p/295062531

In [ ]:
# WARNNG: never do this!

import ctypes

class IntStruct(ctypes.Structure):
    _fields_ = [("ob_refcnt", ctypes.c_long),
                ("ob_type", ctypes.c_void_p),
                ("ob_size", ctypes.c_ulong),
                ("ob_digit", ctypes.c_long)]
    
    def __repr__(self):
        return ("IntStruct(ob_digit={self.ob_digit}, "
                "refcount={self.ob_refcnt})").format(self=self)

c113 = ctypes.c_long(113)
iptr = IntStruct.from_address(id(113))
print(f"113 == 4 is {113 == 4}")
print(f"id(4) = {id(4)}, id(113) = {id(113)}")

# be careful, remember restore the value, or restart the interpreter
iptr.ob_digit = 4  # now Python's 113 contains a 4!
print(f"113 == 4 is {113 == 4}")
print(f"id(4) = {id(4)}, id(113) = {id(113)}")

# restore the value
iptr.ob_digit = c113
print(f"113 == 4 is {113 == 4}")
print(f"id(4) = {id(4)}, id(113) = {id(113)}")

In [ ]:
import numpy

## `id`

In [ ]:
str1_addr = id('abc')
str2_addr = id('abc')

In [ ]:
print(f"str1 addr: {str1_addr}, str2 addr: {str2_addr}")
str1_addr == str2_addr

In [ ]:
import ctypes
import gc
gc.disable()

class Object(ctypes.Structure):
    _fields_ = [("ob_refcnt", ctypes.c_long)]

In [ ]:
l = []
l.append(l)

In [ ]:
l_addr = id(l)

In [ ]:
l_addr

In [ ]:
del l

In [ ]:
Object.from_address(l_addr).ob_refcnt

# Packages in Python

## os, sys

In [ ]:
import os
import sys

# walk
# path
# join
for pth, dirs, files in os.walk('.'):
    print(pth, dirs, files)
    break

## signal

In [ ]:
import signal
from time import sleep

gframe = None
def handler(sig_num, frame):
    global gframe
    print('Sig received with number %d'%sig_num)
    gframe = frame

signal.signal(signal.SIGINT, handler)

print('Start ...')
signal.raise_signal(2)
print('End ...')

In [ ]:
gframe

## ipyparallel

TODO

## numpy

In [ ]:
import numpy as np

In [ ]:
np.isinf(np.inf) or np.isnan

## psutil

In [ ]:
import os
import psutil
pid = os.getpid()
python_process = psutil.Process(pid)
memoryUse = python_process.memory_info()

In [ ]:
memoryUse

## Json

```python
import base64
import json
import numpy as np

class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, complex):
            return str(obj)
        
        return json.JSONEncoder(self, obj)

json._default_encoder = MyEncoder()
```

# Onedrive API

## Download from onedrive

In [ ]:
share_url = 'https://1drv.ms/u/s!Au1wcoQGYu6djJofAu3qVd577D-xgg?e=wASCui'

In [ ]:
import base64
def create_onedrive_directdownload (onedrive_link):
    data_bytes64 = base64.b64encode(bytes(onedrive_link, 'utf-8'))
    data_bytes64_String = data_bytes64.decode('utf-8').replace('/','_').replace('+','-').rstrip("=")
    resultUrl = f"https://api.onedrive.com/v1.0/shares/u!{data_bytes64_String}/root/content"
    return resultUrl

In [ ]:
create_onedrive_directdownload(share_url)

## Uploader

In [ ]:
# file: uploader
import os
import requests
import tqdm
import click

# Here, we get the token from https://developer.microsoft.com/en-us/graph/graph-explorer
# and save it in file access_token
# TODO: get the token automaticly.
def load_access_token(path=None):
    if path is None:
        path = os.getcwd()
    with open(os.path.join(path, "access_token"), "r") as f:
        access_token = f.readline().strip('\n')

    return access_token


def upload(file_to_upload, file_name, access_token, unit=1):
    local_name = file_to_upload

    if file_name is None:
        file_name = os.path.basename(local_name)


    request_body = {
    }

    base_url = "https://graph.microsoft.com/v1.0"
    # folder_id = "01VGN2QX6TWD75CHGPGRG2UCZAOOHFOKEM"

    url_put = base_url + f"/me/drive/root:/{file_name}:/createUploadSession"

    headers = {
        "Authorization": "Bearer " + access_token
    }

    response_upload_session = requests.post(
        url_put, headers=headers, json=request_body
    )

    try:
        upload_url = response_upload_session.json()['uploadUrl']
    except Exception as e:
        raise e

    with open(local_name, "rb") as upload:
        total_file_size = os.path.getsize(local_name)
        chunk_size = 327680*unit
        chunk_number = total_file_size // chunk_size
        chunk_leftover = total_file_size - chunk_size * chunk_number
        counter = 0
        
        bar = tqdm.tqdm(total=chunk_number + 1, 
                        desc="upload")

        while True:
            chunk_data = upload.read(chunk_size)
            start_index = counter * chunk_size
            end_index = start_index + chunk_size

            if not chunk_data:
                break

            if counter == chunk_number:
                end_index = start_index + chunk_leftover

            upload_headers = {
                "Content-Length": f'{chunk_size}',
                "Content-Range": f'bytes {start_index}-{end_index-1}/{total_file_size}'
            }

            chunk_data_upload_status = requests.put(
                upload_url, 
                headers=upload_headers,
                data=chunk_data)
            # print('Upload Progress: {0}'.format(chunk_data_upload_status.json()['nextExpectedRanges']))
            bar.update()

            counter += 1
        bar.close()
        
    requests.delete(upload_url)


@click.command()
@click.option('--token_path', default=None, help='access_token path')
@click.option('--unit', default=16, help='access_token path')
@click.option('--name', default=None, help='remote file name')
@click.argument('file_to_upload')
def main(file_to_upload, name, token_path, unit):
    token = load_access_token(token_path)
    print(token)
    upload(file_to_upload, name, token, unit)

# if __name__ == '__main__':
#     main()

# Jupyter notes

## Creating Custom Templates for nbconvert

https://nbconvert.readthedocs.io/en/latest/customizing.html